# Legal assistant explorer

Copyright 2025, Denis Rothman

This notebook serves as an interactive explorer for the **Legal Compliance Assistant**, an advanced multi-agent system equipped with essential production safeguards like a **two-stage content moderation protocol**

You will use a series of "Control Decks" to test the engine's core capabilities in a legal context, including high-fidelity RAG, context reduction, and grounded reasoning.

Crucially, you will also explore the system's realistic limitations through "limit tests" designed to simulate the real-world "headaches" that arise when AI confronts complex and ambiguous organizational challenges. This hands-on experience demonstrates how to apply, validate, and understand the boundaries of a production-ready Context Engine.

*Note* Make sure to run `Data_Ingestion.ipynb` first to ingest the data for this notebook.

# I. Inititalization

## GitHub

In [1]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/registry.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1823  100  1823    0     0   5553      0 --:--:-- --:--:-- --:--:--  5557
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6479  100  6479    0     0  18236      0 --:--:-- --:--:-- --:--:-- 18199
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9440  100  9440    0     0  30861      0 --:--:-- --:--:-- --:--:-- 30950
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3611  100  3611    0     0   6276      0 --:--:-- --:--:-- --:--:--  6269
  % Total    % Received % Xferd  Average Speed   Tim

## Installation and client setup

In [2]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [3]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [4]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
import json  # <--- Added for type handling
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active):
    """
    Runs the context engine, now with robust type handling for moderation.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        # Goal is always a string, so this is safe
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")

        # HANDLE NON-STRING OUTPUTS ---
        # If the agent returns a dictionary (JSON), convert it to a string first.
        text_to_check = result
        if isinstance(result, (dict, list)):
            try:
                text_to_check = json.dumps(result, indent=2)
            except TypeError:
                text_to_check = str(result)
        # ------------------------------------------

        moderation_report = helpers.helper_moderate_content(text_to_moderate=text_to_check, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        # If result is still a dict (and wasn't redacted), display it nicely
        if isinstance(result, dict):
             display(Markdown(f"```json\n{json.dumps(result, indent=2)}\n```"))
        else:
             display(Markdown(str(result)))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\n\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)

## Control Deck configuration

In [5]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [6]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the Non-Disclosure Agreement."

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment': 4

```json
{
  "summary": "I don\u2019t see the actual NDA text included\u2014just a placeholder (\u201cThe full text of the Non-Disclosure Agreement (NDA) that the user wants summarized.\u201d). I need the agreement\u2019s content to give you an accurate, tailored summary.\n\nPlease paste the full NDA (or upload a clear image/PDF), and I\u2019ll summarize it along the requested points:\n\n1. What counts as \u201cConfidential Information\u201d  \n2. What the receiving party must do (and not do)  \n3. Allowed uses and who it can be shared with  \n4. Information that is NOT protected (exclusions)  \n5. How long the NDA lasts and which obligations survive  \n6. Rules on returning or destroying information  \n7. Remedies for breach (including any right to injunctive relief)  \n8. Any special clauses (non-solicit, non-compete, governing law, jurisdiction, etc.)"
}
```



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 21.09 seconds
Execution Steps:
[ { 'agent': 'Summarizer',
    'output': { 'summary': 'I don’t see the actual NDA text included—just a '
                           'placeholder (“The full text of the Non-Disclosure '
                           'Agreement (NDA) that the user wants summarized.”). '
                           'I need the agreement’s content to give you an '
                           'accurate, tailored summary.\n'
                           '\n'
                           'Please paste the full NDA (or upload a clear '
                           'image/PDF), and I’ll summarize it along the '
                           'requested points:\n'
                           '\n'
                           '1. What counts as “Confidential Information”  \n'
                           '2. What the receiving party must do (and not '
                           'do)  \n'
                           '3.

In [7]:
#@title CONTROL DECK TEMPLATE 1: High-Fidelity RAG

# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

# === CONTROL DECK 1: High-Fidelity RAG in a Legal Context ===
goal = "What are the key confidentiality obligations in the Service Agreement v1, and what is the termination notice period? Please cite your sources."

# === CONTROL DECK 1 (LIMIT TEST): Sanitization of Legal Testimony ===
#goal = "What did Mr. Smith advise his client regarding the assets?"

# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


## Confidentiality Obligations

**High-level summary**  
Both parties must keep all proprietary information disclosed during the term of the Agreement confidential and may not share it with third parties without prior written consent. The clause does not state any exceptions or a post-termination survival period.

**Bullet list of key obligations with clause numbers**

- **Mutual confidentiality obligation (Section 4)**  
  Both parties “agree to maintain the confidentiality of all proprietary information disclosed during the term of this Agreement.”

- **Non-disclosure to third parties (Section 4)**  
  Proprietary information “shall not be disclosed to any third party without prior written consent.”

- **Third-party / affiliate disclosures require consent (Section 4)**  
  There is no express carve-out for affiliates, subcontractors, or other related parties; such disclosures would be to a “third party” and therefore require prior written consent.

- **No listed exceptions (Section 4)**  
  The clause does not specify common exclusions (e.g., public information, independently developed information, information required to be disclosed by law).

**Any survival period**

- **No explicit survival or duration after termination (Section 4)**  
  The clause applies to information “disclosed during the term of this Agreement,” but there is **no express statement** that confidentiality obligations survive termination or for how long they might continue.

---

## Termination Notice Period

**Length of notice period(s)**

- **30 days written notice (Section 5)**  
  Either party may terminate the Agreement “with thirty (30) days written notice.”

**Different notice periods for convenience vs breach (if applicable)**

- **Termination for convenience / any reason (Section 5)**  
  The language allows either party to terminate on 30 days’ written notice without requiring cause, effectively creating a mutual termination-for-convenience right with a 30-day notice period.

- **Termination for breach (Not specifically addressed)**  
  There is no separate termination-for-breach mechanism, no cure period, and no different notice period specified for breach. The only stated notice requirement is the general 30-day written notice.

**Relevant clause numbers**

- **Section 2 – Term**: Establishes a 12‑month fixed term starting June 1, 2025.  
- **Section 5 – Termination**: Provides the 30‑day written notice requirement for termination by either party.

---

## Sources

- **Section 4 – Confidentiality**  
  - “Both parties agree to maintain the confidentiality of all proprietary information disclosed during the term of this Agreement.”  
  - “Information shall not be disclosed to any third party without prior written consent.”

- **Section 2 – Term**  
  - “This Agreement shall commence on June 1, 2025, and continue for a period of twelve (12) months.”

- **Section 5 – Termination**  
  - “Either party may terminate this Agreement with thirty (30) days written notice.”



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 51.97 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'Below is the requested review of the '
                                       'identified “Service Agreement v1” '
                                       'based solely on the provided texts.\n'
                                       '\n'
                                       '---\n'
                                       '\n'
                                       '## Identification of the Correct '
                                       '“Service Agreement v1”\n'
                                       '\n'
                                       'The relevant document is clearly the '
                                       '**Service Agreement** beginning:\n'
                                       '\n'
                                       '> “This Service Agreement '
                                   

In [8]:
#@title CONTROL DECK TEMPLATE 2: Context Reduction

# 1. Define the Goal: A multi-step task that involves summarizing a large
#    document and then using that summary for a different purpose.
#    - DOMAIN: Any field with large documents (legal, scientific, corporate).
#    - KEY CAPABILITY: Tests the `Summarizer` agent and the engine's ability
#      to perform Context Chaining between the `Summarizer` and the `Writer`.
# goal = "[INSERT YOUR CONTEXT REDUCTION GOAL HERE]"

# === CONTROL DECK 2: Context Reduction for Client Communication ===
goal = "First, summarize the Provider Inc. Privacy Policy. Then, using ONLY the information in that summary, draft a short, client-facing paragraph for a website FAQ that explains our data retention policy in simple, non-legalistic terms."

# === CONTROL DECK 2 (LIMIT TEST): The Vague Objective ===
#goal = "Summarize the service agreement and then write a story about it."


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


We don’t yet have any information about how Provider Inc. keeps or deletes your data. The only available text is the title “Provider Inc. Privacy Policy,” and it does not include details about what data is stored, how long it is kept, or how and when it is removed. Because of this, we can’t describe our data retention practices here. Once the full privacy policy is available, we will clearly explain how long we keep your information and what happens to it over time.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 9.31 seconds
Execution Steps:
[ { 'agent': 'Summarizer',
    'output': { 'summary': 'The provided text only contains the title '
                           '“Provider Inc. Privacy Policy” and no actual '
                           'policy content. There is no information about:\n'
                           '\n'
                           '- What data is stored  \n'
                           '- How long it is retained  \n'
                           '- Reasons for retention  \n'
                           '- Deletion or anonymization practices  \n'
                           '- User rights related to retention  \n'
                           '- Legal or compliance constraints on retention '
                           'periods  \n'
                           '\n'
                           'Without the full policy text, no '
                           'data-retention-related summary can be produced. '

In [9]:
#@title CONTROL DECK TEMPLATE 3: Grounded Reasoning & Hallucination Prevention

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# === CONTROL DECK 3: Grounded Reasoning and Hallucination Prevention ===
goal = "Write a persuasive opening statement for a trial involving a monkey that can fly a rocket."

# === CONTROL DECK 3 (LIMIT TEST): The Ambiguous Request ===
#goal = "Analyze the attached NDA and draft a pleading based on its terms."

# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


The monkey’s name was Harlan.

I stood when they called the case. My knees brushed the underside of the table. Cold wood. An empty glass rattled near my hand. Twelve strangers watched me. Somewhere behind them, behind the paneled walls and fluorescent hum, a rocket waited on a video screen.

“This case is about a monkey who piloted a rocket,” I said, “and the company that put him there.”

My voice sounded too loud. It dropped into the quiet and stayed there.

First, there was the lab.

White walls. Stainless steel tables. The soft hiss of air vents. Harlan in a glass enclosure, his fur a dull brown under the harsh lights. His fingers curled around the bars as men in badges walked past, clipboards pressed to their chests. They wrote down everything. Time. Temperature. Food. Heart rate.

He watched them. They watched him.

Then came the plans.

Paper spread across a long table. Blue lines. Sharp angles. A rocket drawn in careful strokes. There were emails, too. You will see them later. Time stamps. Subject lines. “Simian Pilot Program – Acceleration Schedule.” Words that looked clean. Safe. Neat.

Underneath the words, there was something else. Pressure. A quiet rush. A ticking you could not hear but could feel, like a pulse under skin.

“Rushed judgment in a dangerous experiment.” Those were the words that held the whole thing together. Every fact you will hear fits inside that theme. Every sound. Every shadow.

Then came the training.

Harlan in a harness. Straps snug against his chest. A soft beep-beep from a monitor in the corner. His eyes reflected the red lights like tiny coins. A technician tapped a tablet. Numbers jumped. Men in the observation room murmured. Pens clicked. A stopwatch started.

“First, we trained him,” their reports will say.
“Then, we tested him.”
“Finally, we launched.”

You will see the logs. They exist. Black text on white screens. Each session recorded. Start time. End time. Heart rate spikes. Interruptions. Notes in the margins in smaller, hurried letters. “Reaction faster than expected.” “Subject appears agitated.” “Request: delay next stage?”

Those questions sit there in ink. No one erased them. No one answered them, either.

The rocket itself looked clean in the photographs. You will see those, too. A tall, white shape against a dark sky. Lights at its base like little suns. A narrow hatch cut into the side, just big enough.

They fitted him for that hatch. You will hear from the technician who pulled the straps, one by one, and checked for slack. You will hear the small sounds in his memory: the zip of Velcro, the click of metal teeth, the muffled thump of a heartbeat through fabric.

He will tell you he smelled burned insulation. He will tell you he said something about it. He will tell you no one stopped.

The company will tell you this was all planned. Careful. Controlled. They will point to their checklists. Their procedures. Their “safeguards.” They will say they did not rush. That everything was by the book.

The evidence will show something else.

You will see the schedule board, photographed days before the launch. Color-coded lines squeezed together like crowded train tracks. Deadlines side by side. “Investor demo.” “Press event.” “Regulatory review (pending).” Small red flags pushed to the edges. Requests for more time. All of it written down long before anyone sat in this courtroom.

You will see video.

The camera angle is high. It looks down into the capsule. The walls curve tight, close. Wires snake along the sides, taped in place. A small window shows only black. Harlan is strapped into the central seat. His chest rises and falls. You can see it. Slow. Then fast. A gloved hand reaches in, adjusts a buckle, pulls a strap a little tighter.

You will hear the count.

“Ten.”
“Nine.”
“Eight.”

It is calm at first. Voices steady. Numbers falling like small stones into a deep well. Then the sound builds. Engines spinning up. A low growl that becomes a roar. The cameras tremble. Dust lifts in sheets. Shadows jump.

You will not hear anyone say, “Stop.”

There are three things you will learn we had to prove.

We had to show that the company chose speed over safety. That they rushed judgment in a dangerous experiment.

We had to show that they knew the risk. That the warnings were there in their own records, their own voices, long before ignition.

And we had to show that when they strapped Harlan into that rocket, they did so knowing the danger was not under control.

The evidence will walk you through each of those.

You will hear from the safety officer who wrote a memo labeled “URGENT” and never got a reply. You will see that memo. Date. Time. Subject: “G-Force Tolerance Concerns – Simian Pilot.”

You will hear from the engineer who was told to “make the numbers work” before the demonstration, not after.

You will see draft reports with lines struck through. Entire paragraphs gone. The originals still exist. Saved on backup servers. Recovered by someone who thought they might matter someday.

They matter now.

Some of this will sound strange. A monkey. A rocket. Controls he could not possibly understand. But the events themselves are not fantasy. They were written down. Filmed. Logged. Time-stamped by machines that do not care who sits in the capsule.

The evidence will show how carefully everything was documented.

Every training session recorded on overhead cameras. Every lab doorway with a motion sensor. Every keycard swipe trapped in a server rack humming in a locked room. You will see entry logs that place specific people at specific doors at specific times. No guesses. No tricks. Just records.

You will hear where the data came from. Who collected it. Who stored it. Who tried to change it.

You will see the before and after.

Before the launch: emails with subject lines like “We can’t miss this window.” “No delays.” “We need a show.”

After the launch: silence. Gaps. Calls made from personal phones instead of company lines. Things that should have been written down left hanging in the air instead, like smoke.

We will not pretend this story is ordinary. We will not ask you to forget that a living creature was put inside a rocket for a demonstration. But we will ask you to watch how the strange parts connect to very simple facts.

Someone made a choice.
Someone ignored a warning.
Someone turned a key.

At the end, the central question for you will be clear: Did the company allow a dangerous experiment to go forward, too fast and without enough care, putting Harlan at risk when they knew better?

Everything you hear will bend back toward that.

The theme will not change. From the first lab notes to the last video frame, it is the same story: rushed judgment in a dangerous experiment. That is what ties the sounds in the capsule, the shadows on the launchpad, the tremor in a technician’s voice, to the lines of text glowing on computer screens.

By the time you have heard from the witnesses, seen the videos, and traced the path of each decision through the emails and logs, you will be able to answer that question.

Outside this room, the rocket is just another story on a screen.

In here, it is something else.

It is the hiss of air in a sealed capsule. The hard click of a buckle. The sudden, heavy silence between “Three” and “Two.” It is a company that chose not to wait.

When all the evidence has been laid out in front of you—sound by sound, shadow by shadow—you will be in a position to decide what that choice means.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 112.61 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Increase tension and create '
                                  'suspense.", "style_guide": "Use short, '
                                  'sharp sentences. Focus on sensory details '
                                  '(sounds, shadows). Maintain a slightly '
                                  'eerie but age-appropriate tone.", '
                                  '"participants": [{"role": "Agent", '
                                  '"description": "The protagonist '
                                  'experiencing the events."}, {"role": '
                                  '"Source_of_Threat", "description": "The '
                                  'underlying danger or mystery."}], '
                                  '"instruction": "Rewrite the provided facts '
                              